In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
#!pip install pillow,scikit-learn
#import PIL,sklearn
import glob,matplotlib as plt
%matplotlib inline

In [ ]:
#from keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions

In [3]:
from keras.applications.resnet50 import ResNet50,preprocess_input, decode_predictions

In [4]:
img_width, img_height = 224, 224

In [ ]:
#model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [5]:
model = applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [6]:
l=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [7]:
path='train/'
img_train=[]
for i in l:
    img_list = glob.glob(path+str(i)+'**/*.jpg', recursive=True)
    img_train +=img_list
    
path='test/'
img_test= glob.glob(path+'**/*.jpg', recursive=True)
    

In [8]:
len(img_train),len(img_test)

(76505, 21273)

In [9]:
import pandas as pd

In [10]:
label=pd.DataFrame()
label['id']=img_train
label['target']=label.id.apply(lambda x : x.split("/")[1])
label['id']=label.id.apply(lambda x : x.split("/")[2])


In [12]:
import cv2

def read_image(img_path,H,W):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W,H)) # you can resize to  (128,128) or (256,256)
    return img



In [40]:
import gc
gc.collect()

0

In [36]:
%%time
image_lr=[]
train_img=[]
for i in img_train:
    image_lr.append(read_image(i,224,224))
train_img=np.array(image_lr,dtype=np.float16)
train_img=preprocess_input(train_img,data_format='channels_last',mode='tf')

CPU times: user 8min 30s, sys: 9.99 s, total: 8min 40s
Wall time: 8min 45s


In [22]:
del model

In [38]:
%%time
image_lr=[]
test_img=[]
for i in img_test:
    image_lr.append(read_image(i,224,224))
test_img=np.array(image_lr,dtype=np.float16)


CPU times: user 39.9 s, sys: 1.96 s, total: 41.8 s
Wall time: 41.4 s


In [41]:
test_img=preprocess_input(test_img,data_format='channels_last',mode='tf')

In [42]:
train_img.shape,test_img.shape

((76505, 224, 224, 3), (21273, 224, 224, 3))

In [44]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)

test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)
gc.collect()

0

In [45]:
batch_size=16

In [61]:
l=train_img.shape[0]-int(0.1*train_img.shape[0])
l


68855

In [50]:
Y=pd.get_dummies(label.target)
Y.shape

(76505, 15)

In [62]:
x_train=train_img[:l,:,:,:]
y_train=Y[:l]
x_valid=train_img[l:,:,:,:]
y_valid=Y[l:]

In [63]:
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape

((68855, 224, 224, 3), (68855, 15), (7650, 224, 224, 3), (7650, 15))

In [67]:

train_generator = train_datagen.flow(x_train,y_train,
                                    batch_size = batch_size,
                                    shuffle=True)
                                    #class_mode = "categorical")

validation_generator = test_datagen.flow(x_valid,y_valid,
                                         batch_size = batch_size,
                                        shuffle=True)
                                        #class_mode = "categorical")

MemoryError: 

In [ ]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

In [ ]:


#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(15, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)
#model.load_weights('caavo-1.h5')

In [ ]:

# Save the model according to the conditions  
#checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [ ]:
adam = Adam(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch = x_train.shape[0]//batch_size,
epochs = 2,
validation_data = validation_generator,
validation_steps =x_valid.shape[0]//batch_size)
#callbacks = [checkpoint, early])

In [ ]:
# Train the model 
K.set_value(adam.lr, 0.001)

model_final.fit_generator(
train_generator,
steps_per_epoch= x_train.shape[0]//batch_size,
epochs = ,
validation_data = validation_generator,
validation_steps =x_valid.shape[0]//batch_size)
#callbacks = [checkpoint, early])

In [ ]:
pred=predict(test, batch_size=32)

In [ ]:
model.save_weights('caavo-1.h5')